# Funciones utilizadas en el proyecto

- Extraer y transformar datos de un archivo .xml

In [ ]:
def get_prescripcions_esp(bloquePrescripcion: list, especies: list): 
    prescripciones = []
    for prescripcionDict in bloquePrescripcion:
        if not 'dosisrecomendadaespecie' in prescripcionDict:
            continue
        for dosisEspecieDict in prescripcionDict['dosisrecomendadaespecie']: # casi todos los valores eran dict, pero hay algunos str, en este caso los ignoramos y seguimos mirando lso demás
            if type(dosisEspecieDict) == str:
                continue
            if dosisEspecieDict['categoria'] in especies:
                prescripciones.append(prescripcionDict)
                break
    return prescripciones

In [ ]:
def dict_meds(prescripciones: list): 
    Dict_meds_esp = {
        "Medicamento":[],
        "Comercializado":[],
        "Especie de destino":[]
        }
    for prescripcionDict in prescripciones:
        if not 'dosisrecomendadaespecie' in prescripcionDict:
            continue
        for dosisEspecieDict in prescripcionDict['dosisrecomendadaespecie']:
            if type(dosisEspecieDict) == str:
                continue
            Dict_meds_esp['Medicamento'].append(prescripcionDict['nombre_med'])
            Dict_meds_esp['Comercializado'].append(prescripcionDict['comercializado'])
            Dict_meds_esp['Especie de destino'].append(dosisEspecieDict['categoria'])
            break
    return Dict_meds_esp

- Extraer y transformar datos de una web

def crearDriver():
    opciones=Options()
    opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
    opciones.add_experimental_option('useAutomationExtension', False)
    opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
    opciones.add_argument('--start-maximized')         # comienza maximizado
    opciones.add_argument('--incognito')

    driver = webdriver.Chrome(options = opciones)
    return driver

In [ ]:
def extraerRazas(driver, url: str)->list:
       driver.get(url)
    time.sleep(3)


    cookiesButton = driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
    if cookiesButton:
        cookiesButton.click()
        time.sleep(2)

    razas = []

    while True:
        gatos = driver.find_elements(By.CLASS_NAME, 'results-view-name')
        if not gatos:
            break
        razas = razas + [e.text for e in gatos] #extend

        next_page_button = [x for x in driver.find_element(By.CLASS_NAME, 'pager').find_elements(By.CSS_SELECTOR, 'a') if x.get_attribute('title')=='Go to next page']

        if not next_page_button:   # Si no lo encuentra, no hay más páginas
            break
        try:
            next_page_button[0].click()
        except:
            next_page_button[0].click()
        finally:
            time.sleep(3)

    driver.quit()
    return razas